In [1]:
import torch as t
from torch import nn, Tensor
from transformer.config import Config
from transformer.sample_transformer import SampleTransformer, get_log_probs
from transformer.trainer import TransformerTrainingArgs, TransformerTrainer
from transformer.tests import reference_gpt2
import math
from tqdm import tqdm
import datasets
from transformer_lens.utils import tokenize_and_concatenate
from torch.utils.data import DataLoader

c:\Users\saisa\.conda\envs\mi\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded pretrained model gpt2-small into HookedTransformer


In [2]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")

In [3]:
model_cfg = Config(
    debug=False, 
    d_model=256, 
    n_heads=4, 
    d_head=64, 
    d_mlp=1024, 
    n_layers=2, 
    n_ctx=256, 
    d_vocab=reference_gpt2.cfg.d_vocab
)
model = SampleTransformer(model_cfg)

In [4]:
args = TransformerTrainingArgs()

In [5]:
dataset = datasets.load_dataset("NeelNanda/pile-10k", split="train").remove_columns("meta")

In [6]:
print(dataset)
print(dataset[0]['text'][:100])

Dataset({
    features: ['text'],
    num_rows: 10000
})
It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web. Playi


In [7]:
tokenized_dataset = tokenize_and_concatenate(dataset, reference_gpt2.tokenizer, streaming=False, max_length=model.cfg.n_ctx,
                                             column_name="text", add_bos_token=True, num_proc=4)

In [8]:
dataset_dict = tokenized_dataset.train_test_split(test_size=1000)

In [9]:
train_loader = DataLoader(dataset_dict["train"], batch_size=args.batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(dataset_dict["test"], batch_size=args.batch_size, shuffle=False, num_workers=4, pin_memory=True)

In [10]:
first_batch = train_loader.dataset[:args.batch_size]

print(first_batch.keys())
print(first_batch['tokens'].shape)

dict_keys(['tokens'])
torch.Size([16, 256])


In [11]:
model = SampleTransformer(model_cfg).to(device)
args = TransformerTrainingArgs()
trainer = TransformerTrainer(args, model, dataset_dict)
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sasank. Use `wandb login --relogin` to force relogin


accuracy,▁▃▄▅▅▆▇▇▇█
train_loss,█▇▆▅▅▅▆▄▄▅▅▄▃▄▃▂▄▃▄▃▂▃▃▃▂▂▂▃▁▂▃▃▂▃▃▁▃▂▁▂
accuracy,0.27205
train_loss,4.31325
